# Chinese exports by year, country and commodity

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import glob

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
# pd.set_option("display.float_format", lambda x: "%.2f" % x)
pd.options.display.float_format = "${:,.2f}".format

---

### Read processed trade data

In [4]:
df = pd.read_csv(
    "../data/processed/exports_xingjiang_all_countries.csv",
    dtype={
        "commodity_code": str,
        "commodity_chapter": str,
        "commodity_chapter_four": str,
    },
)

In [5]:
df.head()

,year,commodity_code,commodity_chapter,commodity_chapter_desc,commodity_chapter_four,commodity_chapter_four_desc,us_dollar,exported_to,exported_from
0,2019,02071419,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",259200,Kyrgyzstan,Xingjiang Uygur
1,2019,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",3584460,Kyrgyzstan,Xingjiang Uygur
2,2018,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",208000,Kyrgyzstan,Xingjiang Uygur
3,2017,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",106250,Kyrgyzstan,Xingjiang Uygur
4,2020,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",1113600,Kyrgyzstan,Xingjiang Uygur


---

### Exports to countries

In [6]:
xinjiang_exports = df.groupby(["exported_to"]).agg({"us_dollar": sum}).reset_index()

In [32]:
xinjiang_exports[xinjiang_exports["exported_to"].str.contains("Korea")].sort_values(
    "us_dollar", ascending=False
).head(20)

,exported_to,us_dollar
86,Korea Rep.,354603316
122,North Korea,2183979


In [8]:
xinjiang_exports.to_csv(
    "../data/processed/xinjiang_exports_countries_2017_2021.csv", index=False
)

---

### Most exported commodity types

In [9]:
df.head()

,year,commodity_code,commodity_chapter,commodity_chapter_desc,commodity_chapter_four,commodity_chapter_four_desc,us_dollar,exported_to,exported_from
0,2019,02071419,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",259200,Kyrgyzstan,Xingjiang Uygur
1,2019,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",3584460,Kyrgyzstan,Xingjiang Uygur
2,2018,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",208000,Kyrgyzstan,Xingjiang Uygur
3,2017,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",106250,Kyrgyzstan,Xingjiang Uygur
4,2020,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",1113600,Kyrgyzstan,Xingjiang Uygur


In [10]:
xinjiang_exports_types = (
    df.groupby(["commodity_chapter_desc"]).agg({"us_dollar": sum}).reset_index()
)

In [11]:
xinjiang_exports_types.sort_values("us_dollar", ascending=False).head(10)

,commodity_chapter_desc,us_dollar
6,Apparel and clothing accessories; not knitted or crocheted,12361523418
30,Footwear; gaiters and the like; parts of such articles,8530093335
5,Apparel and clothing accessories; knitted or crocheted,7145433906
62,Plastics and articles thereof,4450636812
8,"Articles of leather; saddlery and harness; travel goods, handbags and similar containers; articles of animal gut (other than silk-worm gut)",2424928662
58,Organic chemicals,2136200606
21,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles",2125848899
64,"Preparations of vegetables, fruit, nuts or other parts of plants",2012005412
31,"Fruit and nuts, edible; peel of citrus fruit or melons",1510275242
79,"Textiles, made up articles; sets; worn clothing and worn textile articles; rags",1494637674


In [12]:
xinjiang_exports.to_csv(
    "../data/processed/xinjiang_exports_commodities_2017_2021.csv", index=False
)

### Exports by year

In [13]:
xinjiang_exports_types_year = (
    df.groupby(["year", "commodity_chapter_desc"]).agg({"us_dollar": sum}).reset_index()
)

In [14]:
xinjiang_exports_types_year_pivot = (
    xinjiang_exports_types_year.pivot_table(
        columns="year", index="commodity_chapter_desc", values="us_dollar"
    )
    .fillna(0)
    .reset_index()
)

In [15]:
xinjiang_exports_types_year_pivot.columns = (
    xinjiang_exports_types_year_pivot.columns.astype(str)
)

In [16]:
xinjiang_exports_types_year_pivot["2021"] = xinjiang_exports_types_year_pivot[
    "2021"
].round(0)

In [17]:
xinjiang_exports_types_year_pivot.head()

year,commodity_chapter_desc,2017,2018,2019,2020,2021
0,"Aircraft, spacecraft and parts thereof",$0.00,$0.00,$0.00,$0.00,"$58,259.00"
1,Albuminoidal substances; modified starches; glues; enzymes,"$9,823,866.00","$11,648,087.00","$11,273,195.00","$7,815,793.00","$8,122,353.00"
2,Aluminium and articles thereof,"$64,529,277.00","$116,933,622.00",$0.00,$0.00,"$112,382,667.00"
3,Animal or vegetable fats and oils and their cleavage products; prepared animal fats; animal or vegetable waxes,"$126,051.00","$104,069.00","$82,822.00","$48,143.00","$56,431.00"
4,Animal originated products; not elsewhere specified or included,"$17,981,235.00","$17,393,941.00","$10,039,977.00","$8,030,283.00","$10,654,213.00"


In [18]:
xinjiang_exports_types_year_pivot.sort_values("2021", ascending=False).head(10)

year,commodity_chapter_desc,2017,2018,2019,2020,2021
6,Apparel and clothing accessories; not knitted or crocheted,"$2,845,458,468.00","$2,465,034,316.00","$2,791,886,711.00",$0.00,"$4,259,143,923.00"
21,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles",$0.00,$0.00,$0.00,$0.00,"$2,125,848,899.00"
30,Footwear; gaiters and the like; parts of such articles,"$2,890,864,457.00","$1,958,535,325.00","$1,865,182,639.00",$0.00,"$1,815,510,914.00"
5,Apparel and clothing accessories; knitted or crocheted,"$1,897,409,309.00","$1,540,648,501.00","$1,773,908,024.00","$187,337,449.00","$1,746,130,623.00"
62,Plastics and articles thereof,"$720,671,878.00","$747,512,043.00","$760,309,024.00","$863,236,970.00","$1,358,906,897.00"
54,"Nuclear reactors, boilers, machinery and mechanical appliances; parts thereof",$0.00,$0.00,$0.00,$0.00,"$1,016,381,537.00"
89,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof",$0.00,$0.00,$0.00,$0.00,"$782,275,096.00"
83,"Toys, games and sports requisites; parts and accessories thereof",$0.00,$0.00,$0.00,$0.00,"$767,135,447.00"
32,"Furniture; bedding, mattresses, mattress supports, cushions and similar stuffed furnishings; lamps and lighting fittings, n.e.c.; illuminated signs, illuminated name-plates and the like; prefabricated buildings",$0.00,$0.00,$0.00,$0.00,"$642,189,079.00"
58,Organic chemicals,"$268,769,145.00","$397,794,487.00","$479,526,675.00","$384,541,305.00","$605,568,994.00"


In [19]:
xinjiang_exports_types_year_pivot["mean"] = xinjiang_exports_types_year_pivot[
    ["2017", "2018", "2019", "2020"]
].mean(axis=1)

In [20]:
xinjiang_exports_types_year_pivot["change"] = round(
    (
        xinjiang_exports_types_year_pivot["2021"]
        - xinjiang_exports_types_year_pivot["mean"]
    )
    / xinjiang_exports_types_year_pivot["mean"]
    * 100
)

In [21]:
xinjiang_exports_types_year_pivot[
    xinjiang_exports_types_year_pivot["mean"] > 0
].sort_values("change", ascending=False).head(10)

year,commodity_chapter_desc,2017,2018,2019,2020,2021,mean,change
69,Raw hides and skins (other than furskins) and leather,$0.00,"$136,870.00","$314,638.00","$50,217.00","$821,752.00","$125,431.25",$555.00
52,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin","$22,918,864.00","$13,095,912.00","$13,899,460.00",$0.00,"$68,054,642.00","$12,478,559.00",$445.00
33,Furskins and artificial fur; manufactures thereof,"$2,768,146.00","$1,890,062.00","$8,761,857.00","$10,653,697.00","$27,541,356.00","$6,018,440.50",$358.00
82,"Tools, implements, cutlery, spoons and forks, of base metal; parts thereof, of base metal","$119,773,901.00","$25,152,323.00",$0.00,$0.00,"$123,753,730.00","$36,231,556.00",$242.00
17,Copper and articles thereof,"$2,204,549.00","$3,404,998.00",$0.00,$0.00,"$3,748,719.00","$1,402,386.75",$167.00
43,"Manufactures of straw, esparto or other plaiting materials; basketware and wickerwork","$93,270.00","$487,712.00","$297,469.00","$721,757.00","$1,016,323.00","$400,052.00",$154.00
2,Aluminium and articles thereof,"$64,529,277.00","$116,933,622.00",$0.00,$0.00,"$112,382,667.00","$45,365,724.75",$148.00
6,Apparel and clothing accessories; not knitted or crocheted,"$2,845,458,468.00","$2,465,034,316.00","$2,791,886,711.00",$0.00,"$4,259,143,923.00","$2,025,594,873.75",$110.00
26,"Feathers and down, prepared; and articles made of feather or of down; artificial flowers; articles of human hair","$37,609,389.00","$74,858,938.00","$123,881,490.00",$0.00,"$119,162,940.00","$59,087,454.25",$102.00
92,"Wool, fine or coarse animal hair; horsehair yarn and woven fabric","$2,497,859.00","$1,530,970.00","$2,218,489.00","$7,922,606.00","$6,861,028.00","$3,542,481.00",$94.00


### Exports by year

In [22]:
agg_exports_by_year = df.groupby(["year"]).agg({"us_dollar": sum}).reset_index()

In [23]:
agg_exports_by_year.us_dollar.pct_change() * 100

0       NaN
1   $-12.39
2     $1.94
3   $-69.33
4   $435.32
Name: us_dollar, dtype: float64

In [24]:
agg_exports_by_year.head()

,year,us_dollar
0,2017,13436995662
1,2018,11771866361
2,2019,12000016602
3,2020,3680144438
4,2021,19700596110


In [25]:
agg_exports_by_year.to_csv(
    "../data/processed/xinjiang_exports_agg_years_2017_2021.csv", index=False
)

---

### Cotton exports

In [26]:
xinjiang_52_exports = (
    df[df["commodity_chapter"] == "52"]
    .groupby(["commodity_chapter_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [27]:
xinjiang_52_exports.sort_values("us_dollar", ascending=False).head(20)

,commodity_chapter_desc,exported_to,us_dollar
27,Cotton,Kyrgyzstan,99004238
25,Cotton,Kazakhstan,61101229
66,Cotton,Vietnam,29191946
40,Cotton,Pakistan,22047789
59,Cotton,Tajikistan,16899388
24,Cotton,Italy,6559469
65,Cotton,Uzbekistan,5733623
4,Cotton,Bangladesh,5026244
62,Cotton,United Arab Emirates,3830263
56,Cotton,Spain,2470699


### Tomatoes

In [28]:
xinjiang_0702_exports = (
    df[df["commodity_chapter_four"] == "0702"]
    .groupby(["commodity_chapter_four_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [29]:
xinjiang_0702_exports.sort_values("us_dollar", ascending=False).head()

,commodity_chapter_four_desc,exported_to,us_dollar
0,Tomatoes; fresh or chilled,Kazakhstan,21672786
3,Tomatoes; fresh or chilled,Russia,8842666
1,Tomatoes; fresh or chilled,Kyrgyzstan,3816516
4,Tomatoes; fresh or chilled,Uzbekistan,122927
2,Tomatoes; fresh or chilled,Mongolia,4124


### Polysilicon

In [30]:
xinjiang_280461_exports = (
    df[df["commodity_code"].str.contains("280461")]
    .groupby(["year", "commodity_chapter_four_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [31]:
xinjiang_280461_exports

,year,commodity_chapter_four_desc,exported_to,us_dollar
0,2017,"Hydrogen, rare gases and other non-metals",Pakistan,5893160
1,2019,"Hydrogen, rare gases and other non-metals",Italy,4854
2,2020,"Hydrogen, rare gases and other non-metals",Belgium,494
3,2020,"Hydrogen, rare gases and other non-metals",Croatia,1
4,2020,"Hydrogen, rare gases and other non-metals",Reunion,2
